<a href="https://colab.research.google.com/github/SalehMehrabani/PyTorch-Deep-Learning/blob/main/NN4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Workflow


In [1]:
import torch 
from torch import nn
import matplotlib.pyplot as plt

In [ ]:
device= "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Data: Preparing and Loading


In [4]:
#Create Known parameter
weight = 0.7
bias = 0.3

#Create  dataset
start=0
end=1
step= 0.02

X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

### Splitting Data into Training (Training and Validation set) and Test sets

In [3]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [6]:
def plot_predictions(train_data=X_train,
                     train_labels=y_train,
                     test_data=X_test,
                     test_labels=y_test,
                     predictions=None):
  plt.figure(figsize=(10,7))
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")

  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

  if predictions is not None:
    plt.scatter(test_data, predictions, c="r", s="r", label="Predicitions")

  plt.legend(prop={"size":14})


In [ ]:
plot_predictions()

## Building PyTorch Model

In [7]:
#Create Linear Regression Model Class
#Almost Everything in PyTorch inherent from nn.Module
class LinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.weigths= nn.Parameter(torch.rand(1,
                                           requires_grad=True,
                                           dtype=torch.float))
    self.bias=nn.Parameter(torch.rand(1,
                                      required_grad=True,
                                      dtype=torch.float))
    
    #FORWARD METHOD TO DEFINE THE COMPUTATIONAL IN THE MODULE
    def forward(self, x:torch.Tensor) -> torch.Tensor:
      return self.weights * x + self.bias